# Extracting data

## This file is to extract the noise data which has a different sampling frequency: 125 Hz. 
### This csv files are in the google drive

In [1]:
import pandas as pd
import numpy as np
from functions import *

In [ ]:
def time_f_for_data_extraction(time_array,days_included,hours_included):

    date_array = []
    if days_included: 
        dys = (((time_array/1000)/3600)/24).astype(int)
        hrs = ((((time_array - (dys * 3600*1000*24))/1000)/60)/60).astype(int)
        mins = (((time_array - ((dys * 3600*1000*24) + (hrs*60*60*1000))) /1000)/60).astype(int);
        sec = (((time_array - ((dys * 3600*1000*24) + (hrs*60*60*1000) + (mins * 1000*60))) / 1000)).astype(int)
        mili = ((time_array - ((dys * 3600*1000*24) + (hrs*60*60*1000) + (mins * 1000*60) + (sec*1000)))).astype(int)/1000 
        
        for days, hours, minutes, seconds, milis in zip(dys, hrs,mins,sec,mili):
            if days<10:
                d = '0'+str(days)
            else: d = str(days)
            if hours<10:
                h = '0'+str(hours)
            else: 
                h = str(hours)
            if minutes<10:
                minu = '0'+str(minutes)
            else: 
                minu = str(minutes)
            if seconds<10:
                s = '0'+str(seconds)
            else: 
                s = str(seconds)
        
            date_array = np.append(date_array,d+':'+h+':'+minu+':'+s+':'+str(milis))
        
        """ 
       return (np.array([str(days)+':'+str(hours) + ':'+ str(minutes)+':'+str(seconds)+':'+str(milis) for days, hours, minutes, seconds, milis in zip(dys, hrs,mins,sec,mili)]))
        """
        return date_array
        
    else: 
        
        if hours_included: 

            # to hours 
            hours_residual = (time_array%(60*60*1000))
            hrs = np.asarray((time_array - hours_residual ) / (60*60*1000),dtype = 'int')

            # to minutes 
            minutes_residual = hours_residual%(1000*60)
            mins = np.asarray((hours_residual - minutes_residual)/(60*1000), dtype = 'int')

            #to seconds
            seconds_residual = minutes_residual%1000
            sec = np.asarray((minutes_residual - seconds_residual)/1000, dtype='int')
            
            #to mili 
            mili = np.asarray(seconds_residual, dtype = 'int')

            return (np.array([str(hours)+ ':' +str(minutes)+':'+str(seconds)+':'+str(milis) for hours, minutes, seconds, milis in zip(hrs,mins,sec,mili)]))
            
        else: 
            mins = ((time_array/1000)/60).astype(int);
            sec = ((time_array/1000)%60).astype(int);
            mili = ((time_array - (mins*60*1000 + sec*1000))*100).astype(int)/100 #*100.astype(int)/100 is to round to two decimals

            return (np.array([str(minutes)+':'+str(seconds)+':'+str(milis) for minutes, seconds, milis in zip(mins,sec,mili)]))


In [2]:
df_dict = {}
df2 = pd.read_csv('10.csv')
df2 = df2.drop(['Unnamed: 0'], axis = 1)
df2 = df2.drop(['elapsed_time'], axis = 1)
df2['hour'] = pd.to_numeric(df2['timestamp'].str[11:13], errors='coerce')

In [3]:
df1 = pd.read_csv('09.csv')
df1 = df1.drop(['Unnamed: 0'], axis = 1)
df1['day'] = pd.to_numeric(df1['timestamp'].str[8:10], errors='coerce')
df1.drop(df1[df1['day'] == 8].index, inplace = True)
df1 = df1.drop(['day'], axis=1)
df1 = df1.drop(['elapsed_time'], axis = 1)
df1['hour'] = pd.to_numeric(df1['timestamp'].str[11:13], errors='coerce')
df_dict['09'] = df1[df1['hour'] > 21] 
df_dict['10'] = df2[df2['hour'] < 5 ]
df = pd.concat(df_dict)

df['time'] = ((pd.to_numeric(df['timestamp'].str[8:10], errors='coerce')*24*60*60*1000) + (pd.to_numeric(df['timestamp'].str[11:13], errors='coerce')*60*60*1000) + (pd.to_numeric(df['timestamp'].str[14:16], errors='coerce')*60*1000) + 
                (pd.to_numeric(df['timestamp'].str[17:19], errors='coerce')*1000) + (pd.to_numeric(df['timestamp'].str[19:23], errors='coerce')*1000))
df['shift_time'] = df['time'].shift(-1)
df['dif_time'] = df['shift_time'] - df['time']
df['total_index'] = np.arange(0,len(df))
time_ = df['time'].to_numpy()
values = df['data']
columns = ['total_index', 'data', 'timestamp', 'hour', 'time', 'shift_time', 'dif_time']
df = df[columns]
df


total_index  data                timestamp  hour         time  \
09 9895514            0 -38.0  2021-09-09 22:00:00.004    22  856800004.0   
   9895515            1 -50.0  2021-09-09 22:00:00.012    22  856800012.0   
   9895516            2 -52.0  2021-09-09 22:00:00.020    22  856800020.0   
   9895517            3 -52.0  2021-09-09 22:00:00.028    22  856800028.0   
   9895518            4 -58.0  2021-09-09 22:00:00.036    22  856800036.0   
...                 ...   ...                      ...   ...          ...   
10 2249131      3147089 -52.0  2021-09-10 04:59:59.962     4  881999962.0   
   2249132      3147090 -41.0  2021-09-10 04:59:59.970     4  881999970.0   
   2249133      3147091 -36.0  2021-09-10 04:59:59.978     4  881999978.0   
   2249134      3147092 -32.0  2021-09-10 04:59:59.986     4  881999986.0   
   2249135      3147093 -30.0  2021-09-10 04:59:59.994     4  881999994.0   

             shift_time  dif_time  
09 9895514  856800012.0       8.0  
   9895515  856800020.0       8.0  
   9895516  856800028.0       8.0  
   9895517  856800036.0       8.0  
   9895518  856800044.0       8.0  
...                 ...       ...  
10 2249131  881999970.0       8.0  
   2249132  881999978.0       8.0  
   2249133  881999986.0       8.0  
   2249134  881999994.0       8.0  
   2249135          NaN       NaN  

[3147094 rows x 7 columns]

In [5]:
df_intervals = df[df['dif_time']>8]
interval_end_index = []; interval_ending_time = []; duration_of_interval=[]; interval_starting_time =[]; 
intervals_time_dict = {}; intervals_values_dict = {}; start_timestamp = []
interval_starting_time = [df['time'][0]]

for m in range(0, len(df_intervals)):
    interval_end_index = np.append(interval_end_index, df_intervals['total_index'][m]); interval_end_index = np.asarray(interval_end_index, dtype = 'int')    
    interval_ending_time = np.append(interval_ending_time,df_intervals['time'][m])
    interval_starting_time = np.append(interval_starting_time, df[df['total_index'] == interval_end_index[m]+1]['time'][0])
    duration_of_interval = np.append(duration_of_interval,interval_ending_time[-1] - interval_starting_time[-2])
    
    intervals_time_dict['Interval ' +str(m)] = (df.iloc[df[df['time'] == interval_starting_time[m]]['total_index'][0]:interval_end_index[m]+1]['time']).to_numpy()
    intervals_values_dict['Interval ' +str(m)] = (df.iloc[df[df['time'] == interval_starting_time[m]]['total_index'][0]:interval_end_index[m]+1]['data']).to_numpy()

    start_timestamp = np.append(start_timestamp,df[df['time'] == interval_starting_time[m]]['timestamp'][0])


duration_of_interval = np.append(duration_of_interval, df['time'][-1] - interval_starting_time[-1])
intervals_time_dict['Interval ' +str(m+1)] = (df.iloc[df[df['time'] == interval_starting_time[-1]]['total_index'][0]:df['total_index'][-1]+1]['time']).to_numpy()
intervals_values_dict['Interval ' +str(m+1)] = (df.iloc[df[df['time'] == interval_starting_time[-1]]['total_index'][0]:df['total_index'][-1]+1]['data']).to_numpy()

start_timestamp = np.append(start_timestamp,df[df['time'] == interval_starting_time[-1]]['timestamp'][0])

duration_time = time_f_for_data_extraction(duration_of_interval,False, True)
df_intervals = df_intervals.append(df.iloc[-1], ignore_index = True)
df_intervals['duration mili'] = duration_of_interval; df_intervals['interval duration'] = duration_time
df_intervals['ID'] = np.arange(0,len(df_intervals)) ; df_intervals['start_timestamp'] = start_timestamp

df_intervals = df_intervals.rename(columns={"timestamp": 'end_timestamp'})

columns_intervals = ['total_index', 'data', 'dif_time','start_timestamp', 'end_timestamp', 'hour', 'time', 'interval duration','duration mili', 'ID']
df_intervals = df_intervals[columns_intervals]

/var/folders/1j/mg34b_0x65d86sgz8b5z92th0000gn/T/ipykernel_60741/1403974147.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_intervals = df_intervals.append(df.iloc[-1], ignore_index = True)


In [6]:
df_intervals

,total_index,data,dif_time,start_timestamp,end_timestamp,hour,time,interval duration,duration mili,ID
0,897957,-13.0,16342.0,2021-09-09 22:00:00.004,2021-09-09 23:59:43.660,23,863983660.0,1:59:43:656,7183656.0,0
1,1545916,96.0,2312.0,2021-09-10 00:00:00.002,2021-09-10 01:26:23.666,1,869183666.0,1:26:23:664,5183664.0,1
2,1552828,-66.0,1032.0,2021-09-10 01:26:25.978,2021-09-10 01:27:21.266,1,869241266.0,0:0:55:288,55288.0,2
3,2342332,-46.0,1288.0,2021-09-10 01:27:22.298,2021-09-10 03:12:38.322,3,875558322.0,1:45:16:24,6316024.0,3
4,2344252,-17.0,1032.0,2021-09-10 03:12:39.610,2021-09-10 03:12:54.962,3,875574962.0,0:0:15:352,15352.0,4
5,2368444,-57.0,1032.0,2021-09-10 03:12:55.994,2021-09-10 03:16:08.498,3,875768498.0,0:3:12:504,192504.0,5
6,3145660,81.0,1288.0,2021-09-10 03:16:09.530,2021-09-10 04:59:47.250,4,881987250.0,1:43:37:720,6217720.0,6
7,3147093,-30.0,NaN,2021-09-10 04:59:48.538,2021-09-10 04:59:59.994,4,881999994.0,0:0:11:456,11456.0,7


#### One can choose from this intervals to extract a shorter csv

In [20]:
new_df = df[df['total_index'] > 1545915]
new_df = new_df[new_df['total_index'] < 1552828]
new_df_time = new_df['timestamp']
new_df_time.to_csv('Noise/noise_data_time.csv')
new_df_values = new_df['data']
new_df_values.to_csv('Noise/noise_data_values.csv')

In [19]:
new_df_time

10  647958    2021-09-10 01:26:23.666
    647959    2021-09-10 01:26:25.978
    647960    2021-09-10 01:26:25.986
    647961    2021-09-10 01:26:25.994
    647962    2021-09-10 01:26:26.002
                       ...           
    654865    2021-09-10 01:27:21.226
    654866    2021-09-10 01:27:21.234
    654867    2021-09-10 01:27:21.242
    654868    2021-09-10 01:27:21.250
    654869    2021-09-10 01:27:21.258
Name: timestamp, Length: 6912, dtype: object